In [164]:
import numpy as np
import numpy.random as rand
import numpy.linalg as linalg
import matplotlib.pyplot as plt

# SNR is a range between min and max SNR in dB
def generate_signal(N = 16, K = 4, L = 16, f = 2.4e9, theta_bound = np.pi/2):
    c = 3e8 # speed of light
    wl = c/f # wavelength (lambda)
    d = wl/2 # uniform distance between antennas
        
    # antenna array
    array = np.linspace(0,N-1,N)*d/wl

    theta = rand.rand(K,1) * np.pi - np.pi/2
    
    alpha = (np.random.randn(K,L) + 1j*np.random.randn(K,L))*np.sqrt(1/2)
        
    response = np.exp(1j*2*np.pi*array*np.sin(theta))*np.sqrt(1/N)
    
    Y = np.dot(response.T, alpha)
                
    #Y = np.dot(response.T, alpha).repeat(L, axis=1)
                
    return theta, Y, alpha

def esprit(Y, N = 16, K = 4, f = 2.4e9):
    c = 3e8 # speed of light
    wl = c/f # wavelength (lambda)
    d = wl/2 # uniform distance between antennas
    
    m = len(Y)
    
    # \hat R = E[Y*Y^H] = 1/m * Y*Y^H
    R = Y.dot(Y.conj().T) / m
    
    # ensure matrix is hermitian
    #R = 0.5 * (R + R.conj().T)
    
    # ensure descending order of U, E, V
    U,E,V = linalg.svd(R)
    
    S = U[:, 0:K]
    
    S1 = S[0:m-1,:]
    S2 = S[1:m,:]
    
    # P = S1 \ S2
    P = linalg.pinv(S1) @ S2
    
    # E = eigenvalues, Q = eigenvectors
    eigs,_ = linalg.eig(P)
    
    return np.arcsin(1/np.pi * np.angle(eigs))

N = 5
K = 3
L = 10

theta, X_raw, alpha = generate_signal(N, K, L)

Noise = (np.random.randn(N, L) + 1j*np.random.randn(N, L))*np.sqrt(1/1000)

X = X_raw + Noise

theta_hat = esprit(X_raw, N, K)



print(np.rad2deg(np.sort(theta[:,0])))
print(np.rad2deg(np.sort(theta_hat)))

[-10.58779129  42.86535707  64.37312107]
[-10.58779129  42.86535707  64.37312107]


In [155]:
X.shape

(5, 10)